In [ ]:
import torch
import time
import sys
import json
import os
sys.path.append("../")
sys.path.append("../..")
from torch.nn.functional import log_softmax
from utils.LoadConfig import load_config
from utils.PrepareModel import prepare_GPT2
from utils.cal_score import get_sentence_score

# CLM

In [ ]:
dataset = 'aishell'
with open("/mnt/disk6/Alfred/Rescoring/data/sample/aishell/noLM/data.json") as sample:
    data_json = json.load(sample)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
# device = torch.device('cuda')
device = torch.device('cpu')
model, tokenizer = prepare_GPT2(dataset, device)
model.eval()

In [ ]:
bos = tokenizer.cls_token_id if (tokenizer.cls_token is not None) else tokenizer.bos_token_id
eos = tokenizer.sep_token_id if (tokenizer.sep_token is not None) else tokenizer.eos_token_id
pad = tokenizer.pad_token_id if (tokenizer.pad_token is not None) else 0
# dummy = "[PAD]"
# output = tokenizer(dummy, return_tensors='pt')
# input_ids = output["input_ids"].to(device)
# attention_mask = output['attention_mask'].to(device)
# output = model(
#             input_ids = input_ids,
#             attention_mask = attention_mask
#         ).logits
# output_scores = log_softmax(output, dim = -1)
# score = get_sentence_score(output_scores, input_ids, bos, eos, pad)
# t0 = time.time()
# t1 = time.time()
data = data_json[1]
top_hyp = data['hyps'][0]
output = tokenizer(top_hyp, return_tensors='pt')
input_ids = output["input_ids"].to(device)
attention_mask = output['attention_mask'].to(device)
with torch.no_grad():
    torch.cuda.synchronize()
    t0 = time.time()
    output = model(
        input_ids = input_ids,
        attention_mask = attention_mask
    ).logits
    # torch.cuda.synchronize()
    # t2 = time.time()
    output_scores = log_softmax(output, dim = -1)
    score = get_sentence_score(output_scores, input_ids, bos, eos, pad)
    torch.cuda.synchronize()
    t1 = time.time()
print(f'len:{input_ids.shape[1] - 2}, time:{t1 - t0}')
# print(f'calculate forward only:{t2-t0}')

# for i, data in enumerate(data_json):
#     top_hyp = data['hyps'][0]
#     output = tokenizer(top_hyp, return_tensors='pt')
#     input_ids = output["input_ids"].to(device)
#     attention_mask = output['attention_mask'].to(device)

#     print(f'len:{input_ids.shape[1] - 2}')
    
#     with torch.no_grad():
#         torch.cuda.synchronize()
#         t0 = time.time()
#         output = model(
#             input_ids = input_ids,
#             attention_mask = attention_mask
#         ).logits
#         torch.cuda.synchronize()
#         t2 = time.time()
#         output_scores = log_softmax(output, dim = -1)
#         score = get_sentence_score(output_scores, input_ids, bos, eos, pad)
#         torch.cuda.synchronize()
#         t1 = time.time()

#         print(f'len:{input_ids.shape[1] - 2}, time:{t1 - t0}')
#         print(f'calculate forward only:{t2-t0}')
